In [ ]:
!pip install langchain
!pip install sentence-transformers faiss-cpu
from langchain.text_splitter import CharacterTextSplitter
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.9/70.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 17.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import json
with open('/content/conversation_history.json', 'r') as file:
    data = json.load(file)

In [ ]:
text = [message['text'] for message in data]

In [ ]:
text

['<@U06PS204W0L> has joined the channel',
 '<!channel> please share your views about the product we are working on.',
 'I curated several use cases for our Slack Bot.\nNow I shall create the dataset for the same.',
 'Everyone, please update your work.',
 'Hi Team,\n\nToday me and @Ganesh sir worked on the architecture of Slack API.\nNow, I am building ORM using Django.',
 '<@U06PL3Y8DEK> has joined the channel',
 'Hey everyone,\nKindly share the updates please.',
 'Okay',
 'Sure Hitesh ',
 "Hello Team,\n\nAs I already mentioned we are using this channel for our data creation.\nSo, please try to use the same channel for reporting the updates or if you have any queries or any findings just try to push that on the same channel.\nTry to replace WhatsApp or individual DM's to our slack channel so that we populate some data here to work on.",
 'Hello Team,\n\nCan we have a short meeting after we finish this meeting?',
 '<@U06QF6DJPTJ> has joined the channel',
 'Hello everyone,\n\nWe are usin

In [ ]:
data

[{'subtype': 'channel_join',
  'user': 'U06PS204W0L',
  'text': '<@U06PS204W0L> has joined the channel',
  'inviter': 'U06PL3Y8DEK',
  'type': 'message',
  'ts': '1710827776.077939'},
 {'user': 'U06QF6DJPTJ',
  'type': 'message',
  'ts': '1710772641.448379',
  'client_msg_id': '4c5f06f5-a311-4b89-9d32-4e8d833bac36',
  'text': '<!channel> please share your views about the product we are working on.',
  'team': 'T06Q25C79H7',
  'blocks': [{'type': 'rich_text',
    'block_id': 'HPwjY',
    'elements': [{'type': 'rich_text_section',
      'elements': [{'type': 'broadcast', 'range': 'channel'},
       {'type': 'text',
        'text': ' please share your views about the product we are working on.'}]}]}]},
 {'user': 'U06QF6DJPTJ',
  'type': 'message',
  'ts': '1710757181.390189',
  'client_msg_id': 'e2ee4aa3-6c75-4f6c-a609-84b46406444c',
  'text': 'I curated several use cases for our Slack Bot.\nNow I shall create the dataset for the same.',
  'team': 'T06Q25C79H7',
  'blocks': [{'type': 'ric

In [ ]:
texts = []
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=50)
for message in text:
    chunks = text_splitter.split_text(message)
    texts.extend(chunks)


In [ ]:
print(texts)

['<@U06PS204W0L> has joined the channel', '<!channel> please share your views about the product we are working on.', 'I curated several use cases for our Slack Bot.\nNow I shall create the dataset for the same.', 'Everyone, please update your work.', 'Hi Team,\n\nToday me and @Ganesh sir worked on the architecture of Slack API.\nNow, I am building ORM using Django.', '<@U06PL3Y8DEK> has joined the channel', 'Hey everyone,\nKindly share the updates please.', 'Okay', 'Sure Hitesh', 'Hello Team,', "As I already mentioned we are using this channel for our data creation.\nSo, please try to use the same channel for reporting the updates or if you have any queries or any findings just try to push that on the same channel.\nTry to replace WhatsApp or individual DM's to our slack channel so that we populate some data here to work on.", 'Hello Team,\n\nCan we have a short meeting after we finish this meeting?', '<@U06QF6DJPTJ> has joined the channel', 'Hello everyone,', 'We are using this channe

In [ ]:
model_name = "thenlper/gte-base"
model = SentenceTransformer(model_name)
from sklearn.metrics.pairwise import cosine_similarity

embeddings = model.encode(texts)

embeddings /= np.linalg.norm(embeddings, axis=1, keepdims=True)

##Create a FAISS index

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)


In [ ]:
print("\nEvaluating FAISS index...")
query = "Who worked on the architecture of the Slack API?"
query_embedding = model.encode([query])
scores, indices = index.search(query_embedding, k=5)
print(f"Top 5 results for '{query}':")
for i, (score, idx) in enumerate(zip(scores[0], indices[0])):

    # Calculate cosine similarity from Euclidean distance
    cosine_similarity = 1.0 - (score / (2 * embeddings.shape[1])) ** 0.5

    print(f"Result {i+1} (Similarity: {cosine_similarity:.4f}):")
    print(f"Content: {texts[idx][:200]}...")


Evaluating FAISS index...
Top 5 results for 'Who worked on the architecture of the Slack API?':
Result 1 (Similarity: 0.9903):
Content: Hi Team,

Today me and @Ganesh sir worked on the architecture of Slack API.
Now, I am building ORM using Django....
Result 2 (Similarity: 0.9876):
Content: I curated several use cases for our Slack Bot.
Now I shall create the dataset for the same....
Result 3 (Similarity: 0.9848):
Content: As I already mentioned we are using this channel for our data creation.
So, please try to use the same channel for reporting the updates or if you have any queries or any findings just try to push tha...
Result 4 (Similarity: 0.9835):
Content: <@U06Q3THS0P3> has joined the channel...
Result 5 (Similarity: 0.9835):
Content: <@U06PS204W0L> has joined the channel...


In [ ]:
queries = [
    "Who worked on the architecture of the Slack API?",
    "We are using this channel to generate "
]

# Perform searches for each query
for query in queries:
    query_embedding = model.encode([query])
    scores, indices = index.search(query_embedding, k=2)

    print(f"\nQuery: {query}")
    for i, (score, idx) in enumerate(zip(scores[0], indices[0])):

        # Calculate cosine similarity from Euclidean distance
        cosine_similarity = 1.0 - (score / (2 * embeddings.shape[1])) ** 0.5

        print(f" Result {i+1} (Similarity: {cosine_similarity:.4f}):")
        print(f" Content: {texts[idx]}...")


Query: Who worked on the architecture of the Slack API?
 Result 1 (Similarity: 0.9903):
 Content: Hi Team,

Today me and @Ganesh sir worked on the architecture of Slack API.
Now, I am building ORM using Django....
 Result 2 (Similarity: 0.9876):
 Content: I curated several use cases for our Slack Bot.
Now I shall create the dataset for the same....

Query: We are using this channel to generate 
 Result 1 (Similarity: 0.9881):
 Content: We are using this channel to generate our conversational data to work on.
So I would like everyone to come here and have chats on their task, any doubts.
Use this channel instead of using WhatsApp....
 Result 2 (Similarity: 0.9867):
 Content: As I already mentioned we are using this channel for our data creation.
So, please try to use the same channel for reporting the updates or if you have any queries or any findings just try to push that on the same channel.
Try to replace WhatsApp or individual DM's to our slack channel so that we populate some data

In [ ]:
# Performance metrics
import time
total_query_response_time = 0

for query in queries:
    start_time = time.time()
    query_embedding = model.encode([query])
    scores, indices = index.search(query_embedding, k=3)
    end_time = time.time()

    query_response_time = end_time - start_time
    total_query_response_time += query_response_time

    print(f"\nQuery: {query}")
    print(f"Query Response Time: {query_response_time:.4f} seconds")


# Average performance metrics
average_query_response_time = total_query_response_time / len(queries)


print(f"\nAverage Query Response Time: {average_query_response_time:.4f} seconds")



Query: Who worked on the architecture of the Slack API?
Query Response Time: 0.1010 seconds

Query: We are using this channel to generate 
Query Response Time: 0.0836 seconds

Average Query Response Time: 0.0923 seconds
